# 0. 필요 라이브러리 및 변수, 함수 정의

## import library & package

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import random
import re

from tqdm import tqdm
from itertools import combinations
from collections import deque
from transformers import AutoTokenizer
# from transformers import AutoModel, AutoModelForSequenceClassification
# from datasets import load_dataset, load_metric
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

**BM25: 키워드 기반 랭킹 알고리즘**
- 주어진 쿼리에 대해 문서와의 연관성을 평가하는 랭킹 함수
- Bag-of-words 개념을 사용하여 쿼리에 있는 용어가 각각의 문서에 얼마나 자주 등장하는지를 평가
    - 이때 IDF값을 곱해서 자주 등장하지 않는 단어에 더 큰 가중치를 줌.

## Define function

In [2]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    new_script = deque()
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            new_script.append(line)
            
        new_script = '\n'.join(new_script) # 개행 문자로 합침
        new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
        new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
        new_script = re.sub('/^(file|gopher|news|nntp|telnet|http?|https?|ftps?|sftp):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '<url>', new_script)
    
    return new_script

## Preprocessing

In [3]:
code_folder = "./data/code/"
problem_folders = os.listdir(code_folder) # directory에 있는 폴더 list를 반환

In [4]:
preprocess_scripts = []
problem_nums = []

# 300개 Sample code에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder)) # code/problem000/.py 파일
    problem_num = problem_folder # 문제 번호 폴더명
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preprocess_scripts.append(preprocessed_script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:19<00:00, 15.77it/s]


In [5]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums})

## Create Data Set

### Tokenizer 수행, microsoft에서 개발한 사전 학습 모델인 graphcodebert 사용

In [6]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = 'left'
MAX_LEN = 512

tokens = []
for code in df['code']:
    tokens.append(tokenizer.tokenize(code, max_length=MAX_LEN, truncation=True))

df['tokens'] = tokens # Sample code를 Tokenization해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

In [7]:
# train과 validation data set 분리
train_df, valid_df, train_label, valid_label = train_test_split(
        df,
        df['problem_num'],
        random_state=42,
        test_size=0.1,
        stratify=df['problem_num']
    )

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

**stratify (classification을 다룰 때 매우 중요한 옵션)**
- default=None
- stratify 값을 target으로 지정해주면 각각의 class 비율을 train / validation에 유지해 줌
    - 한 쪽에 쏠려서 분배되는 것을 방지
- 만약 이 옵션을 지정해 주지 않고 분류 문제를 다룬다면, 성능의 차이가 많이 날 수 있음

----------------
#### Create Level 1 dataset
- Random하게 뽑은 Positive pairs와 Negative pairs로 구성
**Train data set**

In [8]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [9]:
total_positive_pairs = []
total_negative_pairs = []

In [10]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code'].to_list()
    other_codes = train_df[train_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 500개 (총 300 * 1000 = 300,000개) 추출
    # negative_pairs 500개 (총 300 * 1000 = 300,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:1000]
    random.shuffle(other_codes)
    other_codes = other_codes[:1000]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:28<00:00, 10.55it/s]


In [11]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
train_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
train_data = train_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
train_data.to_csv('data/train_data_lv1.csv',index=False)

**Validation data set**

In [12]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [13]:
total_positive_pairs = []
total_negative_pairs = []

In [14]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code'].to_list()
    other_codes = valid_df[valid_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 500개 (총 300 * 100 = 30,000개) 추출
    # negative_pairs 500개 (총 300 * 100 = 30,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:100]
    random.shuffle(other_codes)
    other_codes = other_codes[:100]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:03<00:00, 87.87it/s]


In [17]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
valid_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
valid_data = valid_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
valid_data.to_csv('data/valid_data_lv1.csv',index=False)

-------------
#### Create Middle Level dataset
- 유사도가 중간인 코드들의 Positive pairs와 Negative pairs로 이루어짐  
    (사용해본 결과 그닥 성능이 좋지않아 사용하지 않음)
**Training data set**

In [116]:
"""
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

# # code를 토큰화하여 저장, train_df에 저장된 모든 코드들에 대한 token들을 리스트 하나에 저장함
# tokenized_corpus = train_df['tokens'].to_list()
# # 토큰화된 code에 대해 상관관계를 계산, 현재 무작위로 설정된 code에 대해서 수행하기 때문에
# # 상관관계를 계산하는 코드는 같은 문제를 푸는 코드가 아닐 수 있음.
# bm25 = BM25Okapi(tokenized_corpus)

total_positive_pairs = []
total_negative_pairs = []

for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = train_df[train_df['problem_num'] == problem]
    other_solutions = train_df[train_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:10]: # solution_codes: 약 135
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        pos_idx = round(len(pp_scores)/2) # pos_idx: 135 / 2 = 약 68
        neg_idx = round(len(np_scores)/2) # neg_idx: 40450 / 2 = 20,225
        positive_code_ranking = pp_scores.argsort()[::1][pos_idx:]
        negative_code_ranking = np_scores.argsort()[::-1][neg_idx:]
        
        # positive, negative pairs의 길이는 코드당 15개 (총 (300 * 10 * 15) * 2 = 90,000개)
        for i in range(15):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, train_df['code'].iloc[score_idx]))
        
        for i in range(15):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, train_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)
    
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
middle_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
middle_data = middle_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
middle_data.to_csv('data/train_data_Middle.csv',index=False)
"""

100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [1:51:03<00:00, 22.21s/it]


-------------
#### Create High Level dataset
- 유사도가 낮은 Positive pairs와 유사도가 높은 Negative pairs로 이루어짐  
    (역시 성능이 좋지않아 사용하지 않음)
**Training data set**

In [142]:
"""
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

total_positive_pairs = []
total_negative_pairs = []

for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_solutions로 저장
    solutions = train_df[train_df['problem_num'] == problem]
    other_solutions = train_df[train_df['problem_num'] != problem]
    
    positive_pairs = []
    negative_pairs = []
    
    # 같은 문제를 푸는 코드에 대한 토큰을 저장
    pp_tokens = solutions['tokens'].to_list()
    np_tokens = other_solutions['tokens'].to_list()
    ppbm25 = BM25Okapi(pp_tokens)
    npbm25 = BM25Okapi(np_tokens)
    
    # 각각의 코드에 대해 유사도 비교하여 pairs에 추가
    for solution, token in list(zip(solutions['code'], solutions['tokens']))[:10]: # solution_codes: 약 135
        pp_scores = ppbm25.get_scores(token)
        np_scores = npbm25.get_scores(token)
        positive_code_ranking = pp_scores.argsort()[::1]
        negative_code_ranking = np_scores.argsort()[::-1]
        
        # positive, negative pairs의 길이는 코드당 15개 (총 (300 * 10 * 15) * 2 = 90,000개)
        for i in range(15):
            score_idx = positive_code_ranking[i]
            positive_pairs.append((solution, train_df['code'].iloc[score_idx]))
        
        for i in range(15):
            score_idx = negative_code_ranking[i]
            negative_pairs.append((solution, train_df['code'].iloc[score_idx]))

            
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)
    
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
high_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
high_data = high_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
high_data.to_csv('data/train_data_High.csv',index=False)
"""